## Deliverable 3. Create a Travel Itinerary Map.

In [26]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [ ]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

In [ ]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
# Set up the info box for markers - format control

info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Conditions</dt><dd>{Conditions}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b Get the city locations in latitude / longitude
locations = vacation_df[["Lat", "Lng"]]

# Get maximum temp by city to set weights on heat map
max_temp = vacation_df["Max Temp"]

# Create the base map figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

#Creat the heat map layer with city locations / temps as weighting
heat_layer = gmaps.heatmap_layer(locations,weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

# 4a Set up markers for each city using box template
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#Add heat map layer and marker layer to base fiture 
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b Display the figure
fig



In [18]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[(vacation_df["City"] == "Margate") & (vacation_df["Country"] == "GB")]
vacation_end = vacation_df.loc[(vacation_df["City"] == "Margate") & (vacation_df["Country"] == "GB")]
vacation_stop1 = vacation_df.loc[(vacation_df["City"] == "Bridlington") & (vacation_df["Country"] == "GB")]
vacation_stop2 = vacation_df.loc[(vacation_df["City"] == "Atherton") & (vacation_df["Country"] == "GB")] 
vacation_stop3 = vacation_df.loc[(vacation_df["City"] == "Liverpool") & (vacation_df["Country"] == "GB")] 



In [19]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = ((((vacation_start["Lat"]).to_numpy()).item()),(((vacation_start["Lng"]).to_numpy()).item()))
end =   ((((vacation_end["Lat"]).to_numpy()).item()),(((vacation_end["Lng"]).to_numpy()).item()))
stop1 = ((((vacation_stop1["Lat"]).to_numpy()).item()),(((vacation_stop1["Lng"]).to_numpy()).item()))
stop2 = ((((vacation_stop2["Lat"]).to_numpy()).item()),(((vacation_stop2["Lng"]).to_numpy()).item()))
stop3 = ((((vacation_stop3["Lat"]).to_numpy()).item()),(((vacation_stop3["Lng"]).to_numpy()).item()))

In [20]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
start2end = gmaps.directions_layer(start,end, waypoints = [stop1, stop2, stop3],travel_mode='DRIVING',show_markers=True,stroke_color='red')
fig.add_layer(start2end)
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Conditions,Lat,Lng,Hotel Name
0,281,Margate,GB,47.70,broken clouds,51.3813,1.3862,Premier Inn Margate hotel
1,305,Bridlington,GB,46.89,broken clouds,54.0831,-0.1919,Expanse Bridlington Seafront Hotel
2,501,Atherton,GB,48.22,scattered clouds,53.5237,-2.4935,The Mercury Motel
3,226,Liverpool,GB,48.09,few clouds,53.4106,-2.9779,Premier Inn Liverpool City Centre (Moorfields)...
4,281,Margate,GB,47.70,broken clouds,51.3813,1.3862,Premier Inn Margate hotel


In [ ]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Conditions</dt><dd>{Conditions}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
#fig.add_layer(start2end)

# 11b. Display the figure
fig

In [23]:
# Create list of cities to develop printed itinerary
Origin ="26-28 St Peters Rd, Margate CT9 1TH, UK"
Stop1 = "1 Chapel St, Bridlington YO15 2DR, UK"
Stop2 = "68 Market St, Atherton, Manchester M46 0DP, UK"
Stop3 = "1 Islington, Liverpool L3 8EG, UK"
End = "26-28 St Peters Rd, Margate CT9 1TH, UK"

In [ ]:
params = {
    "destinations":"1 Chapel St, Bridlington YO15 2DR, UK" ,
    "origins": "26-28 St Peters Rd, Margate CT9 1TH, UK",
    "key": g_key
}
    
base_url = "https://maps.googleapis.com/maps/api/distancematrix/json"
    # 6e Make request and get the JSON data from the search.
route = requests.get(base_url,destinations=Stop1,origins=Origina,key=gkey).json()
params { "key}

In [28]:

route = requests.get(base_url,destinations=Stop1,origins=Origin,key=g_key).json()


TypeError: request() got an unexpected keyword argument 'destinations'